In [2]:
from datasets import load_dataset

In [11]:
dataset = load_dataset("json", data_files="/app/Files/training_set.jsonl", split="train")

In [12]:

dataset[0]

{'messages': [{'role': 'system',
   'content': 'You are a mathematician who is specialized in linear algebra and also statistics.'},
  {'role': 'user', 'content': 'What is a vector space in linear algebra?'},
  {'role': 'assistant',
   'content': 'A vector space is a collection of vectors where you can add them together and multiply them by scalars, following specific rules.'}]}